# Volume-Based Estimators

The idea of estimators... **citations**

When TARDIS runs, we enter into a loop with two main components: a Monte Carlo iteration occurs, and then the plasma state is updated based on the estimators.

## Theory

To account for the effects of the Monte Carlo packets on the ejecta, TARDIS uses the packets to first determine the average energy density $E$ throughout each shell. To do this, for each shell we sum up the amount of energy each packet has ($\epsilon$) multiplied by how much time it spends in the shell ($\delta t$) relative to our reference time of the simulation ($\Delta t$; this is calculated in [Energy Packet Initialization](../montecarlo/initialization.ipynb)), and then divide by the volume of the shell ($\Delta V$). That is,

$$E=\frac{1}{\Delta V}\sum_i \epsilon_i \frac{\delta t_i}{\Delta t}$$

where we sum over every Monte Carlo packet in the shell (in the implimentation section, we will explain how to account for the fact that the energy of the packet can change throughout the simulation ????????????).

Using this energy density, we can 

## Implimentation

**Mention that each iteration is just running the model and storing the state, plus some other stuff for recording things etc**

## Line Estimators

section

## Code Example

We now show a detailed example of how the plasma is updated using the estimators after a Monte Carlo iteration. First, we import the needed packages and set up a simulation (see [Setting Up the Simulation](../setup/index.rst)):

In [ ]:
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
from tardis.io.atom_data.util import download_atom_data
import numpy as np

# We download the atomic data needed to run this notebook
download_atom_data('kurucz_cd23_chianti_H_He')

In [ ]:
tardis_config = Configuration.from_yaml('tardis_example.yml')
sim = Simulation.from_config(tardis_config)

model = sim.model
plasma = sim.plasma
runner = sim.runner

We show the initial radiative temperature, dilution factor, electron densities, and tau sobolevs in each radial cell (see **link**):

In [ ]:
model.t_rad

In [ ]:
model.w

In [ ]:
plasma.get_value('electron_densities')

In [ ]:
plasma.get_value('tau_sobolevs')

We set the number of packets and we run one iteration of the Monte Carlo simulation:

In [ ]:
N_packets = 10000

# Using the commented out code below, we can also get the number of packets
# from the configuration -- try it out:
#N_packets = tardis_config.no_of_packets

sim.iterate(N_packets)

We now show the values of the three estimators previously mentioned:

In [ ]:
runner.j_estimator

In [ ]:
runner.nu_bar_estimator

In [ ]:
# Because most rows of the j_blue estimatior are partially or mostly
# zero, we show just rows with all nonzero elements
runner.j_blue_estimator[np.all(runner.j_blue_estimator != 0, axis=1)]

We note that the shape of the j_blue estimator and the tau_sobolevs are the same: namely, each contain a value for each possible atomic line transition in each radial cell (as opposed to the other two estimators which just have one value for each cell):

In [ ]:
plasma.get_value('tau_sobolevs').shape, runner.j_blue_estimator.shape

We now advance the state of the simulation based on the estimators, and demonstrate this by showing the four quantities we showed before running the simulation. Compare them with their values above!

In [ ]:
# When advance_state is called, a brief summary of the updated t_rad's
# and w's is given
sim.advance_state();

In [ ]:
model.t_rad

In [ ]:
model.w

In [ ]:
plasma.get_value('electron_densities')

In [ ]:
plasma.get_value('tau_sobolevs')